In [214]:
import pandas as pd
import numpy as np
import altair as alt
from altair import datum
from ipywidgets import interact
from os.path import join

from constants import COLUMNS
from utils import read_latest_daily_counts_df

In [11]:
df = read_latest_daily_counts_df()
SITE_IDS = df[COLUMNS.SITE_ID].unique().tolist()

# Columns
siteid = COLUMNS.SITE_ID
date = COLUMNS.DATE
new_positive_cases = COLUMNS.NEW_POSITIVE_CASES
patients_in_icu = COLUMNS.PATIENTS_IN_ICU
new_deaths = COLUMNS.NEW_DEATHS

df

,siteid,date,new_positive_cases,patients_in_icu,new_deaths
0,FZU,2020-02-05,21,7,-1
1,FZU,2020-02-06,27,8,-1
2,FZU,2020-02-07,35,11,-1
3,FZU,2020-02-08,41,12,5
4,FZU,2020-02-09,41,15,7
...,...,...,...,...,...
1105,FBD,2020-03-27,344,98,51
1106,FBD,2020-03-28,343,108,57
1107,FBD,2020-03-29,350,103,52
1108,FBD,2020-03-30,355,101,49


In [87]:
def apply_theme(base):
    return base.configure_axis(
        labelFontSize=14,
        labelFontWeight=300,
        titleFontSize=18,
        titleFontWeight=300
    ).configure_title(fontSize=18, fontWeight=400, anchor="middle"
    ).configure_legend(
        titleFontSize=18, titleFontWeight=400,
        labelFontSize=14, labelFontWeight=300
    )

In [251]:
end_date = "2020-03-30"

def daily_counts_chart(SiteID):
    category = "category"
    value = "value"
    
    # Mouseover effect
    mouseover = alt.selection_single(on="mouseover", fields=[category]) # nearest=True does not look working
    
    base = alt.Chart(df).transform_fold(
        # Fold three quantitative fields, making three rows from one original row.
        fold=[new_positive_cases, patients_in_icu, new_deaths], 
        as_=[category, value]
    ).encode(
        x=alt.X(
            f"{date}:T", 
            axis=alt.Axis(tickCount=7), 
            title="", 
            scale=alt.Scale(domain=["2020-02-01", "2020-04-20"])
        ),y=alt.Y(
            f"sum({value}):Q", 
            axis=alt.Axis(tickCount=5), 
            title="Number of Patients"
        ),
        color=alt.Color(f"{category}:N", scale=alt.Scale(range=["#5C63A2", "#EC7176", "#F4AB32"])),
        # mouse hover visual effect
        size=alt.condition(~mouseover, alt.value(4), alt.value(6)),
        tooltip=[f"{date}:T", f"{category}:N", f"sum({value}):Q"],
    ).properties(width=700, height=500, title="Number of Possitive Cases, Patients in ICU, and Deaths")
    
    if SiteID != "All":
        base = base.transform_filter(
            alt.FieldEqualPredicate(field=siteid, equal=SiteID)
        )
    
    line = base.mark_line(size=4)
    circle = base.mark_circle(size=50).encode(
        size=alt.condition(~mouseover, alt.value(50), alt.value(80))
    )
    
    text = base.mark_text(align='left', fontWeight=400, dx=10, dy=1).encode(
        text=f"{category}:N",
        size=alt.value(20)
    ).transform_window(
        sort=[alt.SortField(date, order="descending")], 
        rank="rank(date)"
    ).transform_filter(alt.datum.rank == 1)
    
    return apply_theme(line + circle + text).add_selection(mouseover).interactive()

interact(daily_counts_chart, SiteID=["All"] + SITE_IDS, )

interactive(children=(Dropdown(description='SiteID', options=('All', 'FZU', 'FZT', 'FZM', 'FXL', 'FWN', 'FVX',…

<function __main__.daily_counts_chart(SiteID)>

In [290]:
def daily_counts_chart(SiteID):
    
    base = alt.Chart(df).encode(
        x=alt.X(
            f"{date}:T", 
            axis=alt.Axis(tickCount=7), 
            title="", 
            scale=alt.Scale(domain=["2020-02-01", "2020-04-20"])
        ),
        y=alt.Y(
            f"{new_deaths}:Q", 
            axis=alt.Axis(tickCount=5), 
            title="Number of Patients"
        ),
        color=alt.Color(siteid, scale=alt.Scale(range=["lightgray"]), legend=None)
    ).transform_filter(
        alt.FieldGTEPredicate(field=new_deaths, gte=0)
    ).properties(width=700, height=500, title="Number of Deaths")
    
    line = base.mark_line(size=4)
    
    hl_line = base.mark_line(size=4).encode(
        color=alt.value("steelblue")
    ).transform_filter(alt.datum.siteid == SiteID)

    circle = hl_line.mark_circle(size=80)
    
    text = hl_line.mark_text(align='left', fontWeight=400, dx=10, dy=1).encode(
        text=f"{siteid}:N",
        size=alt.value(20),
    ).transform_window(
        sort=[alt.SortField(date, order="descending")], 
        rank="rank(date)"
    ).transform_filter(alt.datum.rank == 1)
    
    return apply_theme(line + text + hl_line + circle).interactive()

interact(daily_counts_chart, SiteID=SITE_IDS, )

interactive(children=(Dropdown(description='SiteID', options=('FZU', 'FZT', 'FZM', 'FXL', 'FWN', 'FVX', 'FUU',…

<function __main__.daily_counts_chart(SiteID)>